## 📚 Prerequisites

Before executing this notebook, make sure you have properly set up your Azure Services, created your Conda environment, and configured your environment variables as per the instructions provided in the [README.md](README.md) file.

## Azure AI Search Orchestration: Indexers, Skillsets, and Skills

Azure AI Search offers advanced search capabilities through a well-coordinated operation of **indexers**, **skillsets**, and **skills**. This hierarchical relationship ensures seamless and efficient data ingestion, enrichment, and searchability.

![image.png](utils/images/orchestration.png)

### Components of Azure AI Search Orchestration

#### 1. Indexers

- **Definition:** Indexers in Azure AI Search automate the process of ingesting, transforming, and loading data from various data sources into an Azure AI search index.
- **Operation:** An indexer connects to a data source, retrieves content, and optionally applies a skillset to transform and enrich the data before loading it into a search index.
- **Supported Data Sources:** Azure Blob Storage, Azure Cosmos DB, Azure SQL Database, and others.
- **Example:** An indexer might ingest documents from Azure Blob Storage, apply a skillset for OCR and entity recognition, and then populate an Azure AI Search index with the enriched content.

#### 2. Skillsets

- **Definition:** A skillset is a collection of skills that execute built-in AI or custom processing over documents retrieved from an external data source. Skillsets are reusable resources in Azure AI Search.
- **Operation:** Skills within a skillset transform the content based on the skill's function. The outputs can be text, structured data, or image descriptions.
- **Example:** A skillset might include an OCR skill for image content, a text translation skill for multilingual support, and an entity recognition skill.

#### 3. Skills

- **Definition:** Skills are operations that transform content. They can be text-based for full-text search or vector-based for vector search.
- **Types:**
  - **Built-in Skills:**  These skills wrap API calls to Azure resources. They are based on pretrained models from Microsoft and can include operations like entity recognition, language detection, and sentiment analysis.
  - **Custom Skills:** Custom code executed externally to the search service, often hosted on an Azure function app. These skills extend the AI enrichment pipeline with custom processing logic.
  - **Utility Skills:** : Internal to Azure AI Search, these skills perform operations like conditional processing, document extraction, and text splitting.
- **Examples:** Text extraction, language detection, entity recognition, and optical character recognition (OCR).

For more information, please take a look at the [documentation here](https://learn.microsoft.com/en-us/azure/search/search-indexer-overview).

In [20]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing


In [21]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType,
    VectorizableTextQuery,
    VectorFilterMode,
)
from azure.search.documents.indexes.models import (
    AzureOpenAIEmbeddingSkill,
    SplitSkill,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswParameters,
    IndexProjectionMode,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SearchField,
    SearchFieldDataType,
    IndexingParameters,
    FieldMappingFunction,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionsParameters,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SimpleField,
    SplitSkill,
    IndexingParametersConfiguration,
    WebApiSkill,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)

In [22]:
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    ExhaustiveKnnAlgorithmConfiguration,
    SemanticPrioritizedFields,
    SemanticConfiguration,
)

In [23]:
service_endpoint = os.getenv("AZURE_AI_SEARCH_SERVICE_ENDPOINT")
index_name = "index-test-5"
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

In [24]:
fields = [
    SearchField(name="path", type=SearchFieldDataType.String, key=True),
    SearchField(name="name", type=SearchFieldDataType.String),
    SearchField(name="url", type=SearchFieldDataType.String),
    SimpleField(name="parent_id", type=SearchFieldDataType.String),
    SimpleField(name="chunk_id", type=SearchFieldDataType.String),
    SearchField(name="chunk", type=SearchFieldDataType.String),
    SimpleField(
        name="enriched", type=SearchFieldDataType.String, searchable=False
    ),  # debugging only
    SearchField(
        name="textVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
    SearchField(name="metadata", type=SearchFieldDataType.String),  # Add this line
]
vector_config = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        ),
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer="myOpenAI",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
            vectorizer="myOpenAI",
        ),
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myOpenAI",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=os.getenv("AZURE_AOAI_API_ENDPOINT"),
                deployment_id=os.getenv("AZURE_AOAI_EMBEDDING_DEPLOYMENT_ID"),
                api_key=os.getenv("AZURE_AOAI_API_KEY"),
            ),
        ),
    ],
)

semantic_config = SemanticConfiguration(
    name="mySemanticConfig",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="content")]
    ),
)

index = SearchIndex(name=index_name, fields=fields, vector_search=vector_config)

In [25]:
index_client = SearchIndexClient(service_endpoint, AzureKeyCredential(key))

In [26]:
index_client.create_or_update_index(index)

In [27]:
# Create a data source
ds_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
container = SearchIndexerDataContainer(name="testretrieval")
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type="azureblob",
    connection_string=os.getenv("AZURE_STORAGE_CONNECTION_STRING"),
    container=container,
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'index-test-5-blob' created or updated


In [28]:
skillset_name = f"{index_name}-skillset"

In [31]:
split_skills = SplitSkill(
    description="A skill that splits pdf",
    name="#1",
    context="/document",
    text_split_mode="pages",
    maximum_page_length=2500,
    page_overlap_length=500,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/content"),
    ],
    outputs=[OutputFieldMappingEntry(name="textItems", target_name="pages")],
)

In [32]:
embedding_skill = AzureOpenAIEmbeddingSkill(
    description="Skill to generate embeddings via Azure OpenAI",
    name="#2",
    context="/document/pages/*",
    resource_uri=os.getenv("AZURE_AOAI_API_ENDPOINT"),
    deployment_id=os.getenv("AZURE_AOAI_EMBEDDING_DEPLOYMENT_ID"),
    api_key=os.getenv("AZURE_AOAI_API_KEY"),
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*"),
    ],
    outputs=[OutputFieldMappingEntry(name="embedding", target_name="textVector")],
)

In [11]:
custom_skill = WebApiSkill(
    description="A custom skill that creates an image vector and description",
    uri="https://myskill.gentlebay-4474176e.westeurope.azurecontainerapps.io/vectorize",
    http_method="POST",
    timeout="PT60S",
    batch_size=4,
    degree_of_parallelism=4,
    context="/document",
    inputs=[
        InputFieldMappingEntry(name="url", source="/document/url"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="embedding", target_name="imageVector"),
        OutputFieldMappingEntry(name="description", target_name="description"),
    ],
)

In [33]:
skillset = SearchIndexerSkillset(
    name=skillset_name,
    description="Skillset to chunk documents and generating embeddings",
    skills=[split_skills, embedding_skill],
)

In [34]:
client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
client.create_or_update_skillset(skillset)
print(f"Skillset '{skillset.name}' created or updated")

Skillset 'index-test-5-skillset' created or updated


In [35]:
from typing import Any, Dict, Optional, Union
from azure.search.documents.indexes.models import (
    BlobIndexerParsingMode,
    BlobIndexerDataToExtract,
    BlobIndexerImageAction,
    BlobIndexerPDFTextRotationAlgorithm,
    IndexingSchedule,
)

In [36]:
from datetime import datetime, timedelta
import pytz

# Create a timedelta object representing an interval of 12 hours
interval = timedelta(hours=12)

# Create a datetime object representing the current time in UTC
start_time = datetime.now(pytz.utc)

# Create an instance of IndexingSchedule
schedule = IndexingSchedule(interval=interval, start_time=start_time)

In [16]:
?IndexingParametersConfiguration

Init signature:
IndexingParametersConfiguration(
    *,
    additional_properties: Optional[Dict[str, Any]] = None,
    parsing_mode: Union[str, ForwardRef('_models.BlobIndexerParsingMode')] = 'default',
    excluded_file_name_extensions: str = '',
    indexed_file_name_extensions: str = '',
    fail_on_unsupported_content_type: bool = False,
    fail_on_unprocessable_document: bool = False,
    index_storage_metadata_only_for_oversized_documents: bool = False,
    delimited_text_headers: Optional[str] = None,
    delimited_text_delimiter: Optional[str] = None,
    first_line_contains_headers: bool = True,
    document_root: Optional[str] = None,
    data_to_extract: Union[str, ForwardRef('_models.BlobIndexerDataToExtract')] = 'contentAndMetadata',
    image_action: Union[str, ForwardRef('_models.BlobIndexerImageAction')] = 'none',
    allow_skillset_to_read_file_data: bool = False,
    pdf_text_rotation_algorithm: Union[str, ForwardRef('_models.BlobIndexerPDFTextRotationAlgorithm')] =

In [17]:
?IndexingParametersConfiguration

Init signature:
IndexingParametersConfiguration(
    *,
    additional_properties: Optional[Dict[str, Any]] = None,
    parsing_mode: Union[str, ForwardRef('_models.BlobIndexerParsingMode')] = 'default',
    excluded_file_name_extensions: str = '',
    indexed_file_name_extensions: str = '',
    fail_on_unsupported_content_type: bool = False,
    fail_on_unprocessable_document: bool = False,
    index_storage_metadata_only_for_oversized_documents: bool = False,
    delimited_text_headers: Optional[str] = None,
    delimited_text_delimiter: Optional[str] = None,
    first_line_contains_headers: bool = True,
    document_root: Optional[str] = None,
    data_to_extract: Union[str, ForwardRef('_models.BlobIndexerDataToExtract')] = 'contentAndMetadata',
    image_action: Union[str, ForwardRef('_models.BlobIndexerImageAction')] = 'none',
    allow_skillset_to_read_file_data: bool = False,
    pdf_text_rotation_algorithm: Union[str, ForwardRef('_models.BlobIndexerPDFTextRotationAlgorithm')] =

In [37]:
indexing_params = IndexingParametersConfiguration(
    parsing_mode="text",
    excluded_file_name_extensions=".jpg, .png",
    indexed_file_name_extensions=".pdf",
    fail_on_unsupported_content_type=True,
    fail_on_unprocessable_document=True,
    index_storage_metadata_only_for_oversized_documents=False,
    first_line_contains_headers=False,
    data_to_extract="contentAndMetadata",
    image_action="none",
    allow_skillset_to_read_file_data=True,
    pdf_text_rotation_algorithm="none",
    execution_environment="standard",
    query_timeout=None,
)

In [38]:
indexer_name = f"{index_name}-indexer"

indexer = SearchIndexer(
    name=indexer_name,
    description="Indexer to index documents and generate description and embeddings",
    skillset_name=skillset_name,
    target_index_name=index_name,
    schedule=schedule,
    parameters=IndexingParameters(
        max_failed_items=-1,
        configuration=indexing_params,
    ),
    data_source_name=data_source.name,
    # Map the metadata_storage_name field to the title field in the index to display the PDF title in the search results
    field_mappings=[
        FieldMapping(
            source_field_name="metadata_storage_path",
            target_field_name="path",
            mapping_function=FieldMappingFunction(name="base64Encode"),
        ),
        FieldMapping(
            source_field_name="metadata_storage_name", target_field_name="name"
        ),
        FieldMapping(
            source_field_name="metadata_storage_path", target_field_name="url"
        ),
    ],
    output_field_mappings=[
        FieldMapping(
            source_field_name="/document/textVector", target_field_name="textVector"
        ),
    ],
)

indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
indexer_result = indexer_client.create_or_update_indexer(indexer)

In [39]:
indexer_client.run_indexer(indexer_name)